In [1]:
import os
import yaml
from torch.utils.data import DataLoader
import argparse

from GeospatialFM.data import get_datasets
from GeospatialFM.models import *
# from utils import load_config
from torchgeo.samplers import RandomGeoSampler
from matplotlib import pyplot as plt

from transformers import TrainingArguments, Trainer
from transformers import AdamW, get_linear_schedule_with_warmup
from GeospatialFM.utils import setup, get_eval_fn
from GeospatialFM.data import *

%load_ext autoreload
%autoreload 2

/home/haozhesi/anaconda3/envs/sat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda:0')

In [3]:
args = {'exp_name': None,
        'config_file': 'GeospatialFM/configs/eurosat.yaml',
        'opts': None, 
        'output_dir': './results/configs/eurosat', 
        'save_config': False}
args = argparse.Namespace(**args)
args

Namespace(exp_name=None, config_file='GeospatialFM/configs/eurosat.yaml', opts=None, output_dir='./results/configs/eurosat', save_config=False)

In [4]:
cfg = setup(args)

In [5]:
training_args = TrainingArguments(**cfg['TRAINER'])
model = get_model(cfg['MODEL'])
model = model.to(device)
train_ds, val_ds, test_ds = get_datasets(cfg['DATASET'])
compute_metrics = get_eval_fn(cfg['DATASET'])

In [6]:
training_args.device

device(type='cuda', index=0)

In [7]:
trainer = Trainer(
    model=model,                # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_ds,    # training dataset
    eval_dataset=val_ds,      # evaluation dataset
    compute_metrics=compute_metrics,
)

In [8]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 160.00 MiB (GPU 3; 47.54 GiB total capacity; 512 bytes already allocated; 84.19 MiB free; 2.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
model.device

AttributeError: 'ViTEncoderDecoder' object has no attribute 'device'